# Prototype Financial Planner

In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import REST, TimeFrame
import numpy as np
from MCForecastTools import MCSimulation
import json
import re
from utils import find_crypto_value, calc_start_date, get_company
from datetime import date


load_dotenv()

True

## Part 1 - Personal Finance Planner
---
### Collect Crypto Prices Using the **requests** library

In [2]:
# We will use default crypto values of 1.2 BTC and 5.3 ETH
my_btc = 1.2
my_eth = 5.3

# Crypto API URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/"

# Store and parse crypto data from alternative.me free API
btc_response = requests.get(btc_url)
btc_data = btc_response.json()
eth_response = requests.get(eth_url)
eth_data = eth_response.json()




In [3]:
my_btc_value = find_crypto_value(my_btc, btc_data, 2)
my_eth_value = find_crypto_value(my_eth, btc_data, 2)
print(f"The current value of your {my_btc} BTC is ${my_btc_value:0.2f}")
print(f"The current value of your {my_eth} ETH is ${my_eth_value:0.2f}")

The current value of your 1.2 BTC is $50707.20
The current value of your 5.3 ETH is $223956.80


### Collect Investments Data Using Alpaca: SPY (stocks) and AGG (bonds)

In [4]:
# Set user share totals for SPY and AGG
my_spy = 50
my_agg = 200

In [5]:
# Set Alpca API key and secret key, then insantiate an API object
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECURITY_KEY")

api = tradeapi.REST(alpaca_api_key,alpaca_secret_key,api_version='v2')


In [6]:
# Format current date as ISO format
today = pd.Timestamp("2020-07-14", tz="America/New_York").isoformat()

# Set the tickers
tickers = ["AGG", "SPY"]



# (use a limit=1000 parameter to call the most recent 1000 days of data)
start_date = calc_start_date(5)
df_portfolio = api.get_bars(tickers, TimeFrame.Day, start_date, limit=1000).df

# Preview DataFrame
df_portfolio

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2017-03-20 04:00:00+00:00,107.86,108.0500,107.8401,108.05,1635249,8198,107.961544,AGG
2017-03-21 04:00:00+00:00,107.93,108.2700,107.9300,108.27,2049680,12024,108.175725,AGG
2017-03-22 04:00:00+00:00,108.44,108.5400,108.3600,108.42,2643641,11111,108.455049,AGG
2017-03-23 04:00:00+00:00,108.44,108.5100,108.2635,108.45,2071485,10901,108.417635,AGG
2017-03-24 04:00:00+00:00,108.35,108.5100,108.3100,108.36,3374871,11229,108.398485,AGG
...,...,...,...,...,...,...,...,...
2021-03-03 05:00:00+00:00,114.80,114.8300,114.6000,114.69,10317816,27664,114.725856,AGG
2021-03-04 05:00:00+00:00,114.79,114.8000,114.2600,114.36,5953649,22047,114.499150,AGG
2021-03-05 05:00:00+00:00,114.17,114.3197,114.0500,114.28,8675825,23143,114.237029,AGG


'2017-03-19'